In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

# Steps ch1: 
 __Data Prep__  
- Review datasets 
- Identify common variables 
- Do data cleansing 
- Do inputing & transformation to ensure equal units and time horizons 
--Freq: Austin (Daily), Dheli (Hourly), Szeged (Hourly), Madrid (Daily) 


__Comparison__ 
- Definition of success (How to determine tail vs. headwind) 
- Best location would have: 
- - Primary: Low tailwind; 
- - Secondary:  Low rainfall; 3) Avg temp < 95F; 4) Good visibility  
- Good way to determine would be plots of: 
- Min/Max/Std dev ect + visual for
- Wind magnitude (and trend?) 
 - Daily Temp per location (and trend?) 
- Average Daily Rainfall per location (and trend?)
- Daily Visibility Index per location (and trend?)

__ List of assumptions made __ 
- Using means for inputing missing data points for windspeed, rainfall, temperature, visibility 


__Outstanding Questions__
- Q: Austin dataset does not appear to have a wind direction ? 
-- A:  That is expected. 
- Q: How determine tail vs. head wind? 
-- A: Wind magnitude more importance. 
-- A:  More second challenge. 
- Q: Can I use external data if needed? 
-- A: Totally fine if you want. 

In [180]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.style.use('seaborn')


In [187]:
df_a_raw = pd.read_csv('../input/austin-weather/austin_weather.csv')
df_d_raw = pd.read_csv('../input/delhi-weather-data/testset.csv')
df_s_raw = pd.read_csv('../input/szeged-weather/weatherHistory.csv')
df_m_raw = pd.read_csv('../input/weather_madrid_lemd_1997_2015.csv/weather_madrid_LEMD_1997_2015.csv')

In [188]:
df_a_raw.columns = [c.lower() for c in list(df_a_raw.columns)]
df_d_raw.columns = [c.lower() for c in list(df_d_raw.columns)]
df_s_raw.columns = [c.lower() for c in list(df_s_raw.columns)]
df_m_raw.columns = [c.lower().strip() for c in list(df_m_raw.columns)]

# 1. Data preparation section

## Preping Austin Dataset 

In [189]:
df_a_1 = df_a_raw[['date','temphighf','tempavgf','templowf','precipitationsuminches','windhighmph','windavgmph','windgustmph','visibilityhighmiles','visibilityavgmiles','visibilitylowmiles']]

#Mean value imputing 
df_a_1.loc[df_a_1['precipitationsuminches'] == 'T', 'precipitationsuminches'] = 0
for c in list(df_a_1.columns)[1:]: 
    #print(c)
    newCol = pd.to_numeric(df_a_1[c], errors='coerce')
    df_a_1[c+'_clean']= newCol.fillna(np.mean(newCol))

In [190]:
# Unit conversion 
df_a_1['temphighc'] = (df_a_1['temphighf_clean']-32)*(5.0/9)
df_a_1['tempavgc'] = (df_a_1['tempavgf_clean']-32)*(5.0/9)
df_a_1['templowc'] = (df_a_1['templowf_clean']-32)*(5.0/9)
df_a_1['windhigk'] = df_a_1['windhighmph_clean']*1.609344
df_a_1['windavgk'] = df_a_1['windavgmph_clean']*1.609344
df_a_1['windgustk'] = df_a_1['windgustmph_clean']*1.609344
df_a_1['vishighk'] = df_a_1['visibilityhighmiles_clean']*1.609344
df_a_1['visavgk'] = df_a_1['visibilityavgmiles_clean']*1.609344
df_a_1['vislowk'] = df_a_1['visibilitylowmiles_clean']*1.609344
df_a_1['prcpcm'] = df_a_1['precipitationsuminches_clean']*2.54

In [191]:
df_a_2=df_a_1[['date','temphighc','tempavgc','templowc','windhigk','windavgk','windgustk','vishighk','visavgk','vislowk','prcpcm']]
df_a_2['location']='austin'
df_a_2.head()

In [215]:
df_a_2.to_csv('./out/austin_clean.csv')

## Preping Madrid dataset

In [267]:
df_m_1=df_m_raw[['cet', 'max temperaturec', 'mean temperaturec', 'min temperaturec', 'max visibilitykm', 'mean visibilitykm', 'min visibilitykm', 'max wind speedkm/h', 'mean wind speedkm/h', 'max gust speedkm/h', 'precipitationmm', 'winddirdegrees']]
for c in list(df_m_1.columns)[1:]: 
    #print(c)
    newCol = pd.to_numeric(df_m_1[c], errors='coerce')
    df_m_1[c+'_clean']= newCol.fillna(np.mean(newCol))

In [268]:
#data.rename(columns={'gdp':'log(gdp)'}, inplace=True)
df_m_1.rename(columns={
    'cet':'date',
'max temperaturec_clean': 'temphighc',
'mean temperaturec_clean': 'tempavgc',
'min temperaturec_clean': 'templowc',
'max wind speedkm/h_clean': 'windhigk',
'mean wind speedkm/h_clean': 'windavgk',
'max gust speedkm/h_clean': 'windgustk',
'max visibilitykm_clean': 'vishighk',
'mean visibilitykm_clean': 'visavgk',
'min visibilitykm_clean': 'vislowk',
'precipitationmm_clean': 'prcpcm',
'winddirdegrees_clean': 'winddir',
    },inplace=True)
            

In [269]:
df_m_1['date2'] = [dtm.datetime.strptime(c, "%Y-%m-%d") for c in df_m_1['date']] 
df_m_1['date'] = df_m_1['date2'].dt.strftime("%Y-%m-%d")

In [270]:
df_m_2=df_m_1[['date','temphighc','tempavgc','templowc','windhigk','windavgk','windgustk','vishighk','visavgk','vislowk','prcpcm','winddir']]
df_m_2['location']='madrid'
df_m_2.head()

In [271]:
df_m_2.to_csv('./out/madrid_clean.csv')

## Preping Dheli data

In [195]:
import datetime as dtm 
import time as tm 

In [196]:
df_d_1 = df_d_raw[['datetime_utc',' _tempm',' _precipm',' _vism',' _wdird',' _wgustm',' _wspdm']]

In [197]:
#df_d_1['date']=df_d_1['datetime_utc'].str.slice(0,4)
df_d_1['date']=df_d_1['datetime_utc'].str.slice(0,4)+'-'+df_d_1['datetime_utc'].str.slice(4,6)+'-'+df_d_1['datetime_utc'].str.slice(6,8)

In [198]:
grouped = df_d_1.groupby('date')
g1 = grouped[' _tempm'].agg([np.max, np.mean, np.min]).reset_index()
g1.columns=['date','temphighc','tempavgc','templowc']

In [199]:
g2 = grouped[' _vism'].agg([np.max, np.mean, np.min]).reset_index()
g2.columns=['date','vishighk','visavgk','vislowk']

In [200]:
g3 = grouped[' _wspdm'].agg([np.max, np.mean]).reset_index()
g3.columns=['date','windhigk','windavgk']

In [201]:
g4 = grouped[' _precipm'].agg([np.sum]).reset_index()
g4.columns=['date','prcpcm']

In [202]:
df_d_2 = pd.merge(g1,g2,on=['date'])
df_d_2 = pd.merge(df_d_2,g3,on=['date'])
df_d_2 = pd.merge(df_d_2,g4 ,on=['date'])
df_d_2['location']='delhi'
df_d_2.head()

In [214]:
df_d_2.to_csv('./out/delhi_clean.csv')

## Preping Szeged Data

In [203]:
list(df_s_raw.columns)

In [204]:
df_s_1=df_s_raw[['formatted date',
 'temperature (c)',
 'wind speed (km/h)',
 'wind bearing (degrees)',
 'visibility (km)']]

In [205]:
#df_s_1['date']= df_s_1['formatted date'].str.slice(0,4)
df_s_1['date']=df_s_1['formatted date'].str.slice(0,10)

In [206]:
grouped = df_s_1.groupby('date')
g1 = grouped['temperature (c)'].agg([np.max, np.mean, np.min]).reset_index()
g1.columns=['date','temphighc','tempavgc','templowc']

In [207]:
g2 = grouped['visibility (km)'].agg([np.max, np.mean, np.min]).reset_index()
g2.columns=['date','vishighk','visavgk','vislowk']

In [208]:
g3 = grouped['wind speed (km/h)'].agg([np.max, np.mean]).reset_index()
g3.columns=['date','windhigk','windavgk']

In [209]:
df_s_2 = pd.merge(g1,g2,on=['date'])
df_s_2 = pd.merge(df_s_2,g3,on=['date'])
df_s_2['location']='szeged'
df_s_2.head()

In [213]:
df_s_2.to_csv('./out/szeged_clean.csv')

# 2. Analysis section

__Comparison__ 
- Definition of success (How to determine tail vs. headwind) 
- Best location would have: 
- - Primary: Low tailwind; 
- - Secondary:  Low rainfall; 3) Avg temp < 95F; 4) Good visibility  
- Good way to determine would be plots of: 
- Min/Max/Std dev ect + visual for
- Wind magnitude (and trend?) 
 - Daily Temp per location (and trend?) 
- Average Daily Rainfall per location (and trend?)
- Daily Visibility Index per location (and trend?)

In [272]:
df_main = pd.DataFrame()
df_main = df_main.append(df_a_2)
print (df_main.shape)
df_main = df_main.append(df_m_2)
print (df_main.shape)
df_main = df_main.append(df_d_2)
print (df_main.shape)
df_main = df_main.append(df_s_2)
print (df_main.shape)

In [273]:
df_main.head()

### Overall Stats 

In [274]:
df_main[df_main.location=='austin'].describe()

In [229]:
df_main[df_main.location=='delhi'].describe()

In [275]:
df_main[df_main.location=='madrid'].describe()

In [276]:
df_main[df_main.location=='szeged'].describe()

In [253]:
#Very rough outlier removal 
print(str(len(df_main)))
df_main= df_main[df_main.windavgk<50]
print(str(len(df_main)))
df_main= df_main[df_main.visavgk<50]
print(str(len(df_main)))

### Average Temperature distribution

In [254]:
#df_main[['tempavgc', 'location']].hist(by='location', figsize=[16, 10])
df_main[['tempavgc', 'location']].hist(by='location', figsize=[16, 10],sharex=True)

### Average wind speed 

In [255]:
#df_main[['windavgk', 'location']].hist(by='location', figsize=[16, 10])
df_main[['windavgk', 'location']].hist(by='location', figsize=[16, 10],sharex=True)

### Average Visibility distribution

In [256]:
#df_main[['visavgk', 'location']].hist(by='location', figsize=[16, 10])
df_main[['visavgk', 'location']].hist(by='location', figsize=[16, 10],sharex=True)

### Average daily precipitation

In [251]:
df_main[['prcpcm', 'location']].hist(by='location', figsize=[16, 10])
#df_main[['prcpcm', 'location']].hist(by='location', figsize=[16, 10],sharex=True)